In [1]:
from htmd import *

Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049




Using Anaconda Cloud api site https://api.anaconda.org


New HTMD version (1.1.0) is available. You are currently on (1.0.26). Use 'conda update htmd' to update to the new version.


In [2]:
mol = Molecule('3PTB')
mol.filter("protein")

2016-06-27 19:02:03,755 - htmd.molecule.molecule - INFO - Attempting PDB query for 3PTB
2016-06-27 19:02:04,748 - htmd.molecule.molecule - INFO - Removed 72 atoms. 1629 atoms remaining in the molecule.


In [3]:
copied = mol.copy()

In [5]:
copied.filter('resname ARG LYS ASP GLU HIS CYS TYR')

2016-06-27 19:02:59,117 - htmd.molecule.molecule - INFO - Removed 1175 atoms. 454 atoms remaining in the molecule.


In [6]:
rid = np.unique(copied.get('resid')) # ia.resid also works!
rn = copied.get('resname', "name CA")

In [37]:
### This is for the ND1 atom
for resname, resid in zip(rn, rid):
    if resname == "HIS":
        atomlist = [mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name ND1 and resid "+str(resid)), mol.get("name", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name ND1 and resid "+str(resid))]
        print (atomlist[0], "resid = ",resid)
        print(atomlist[1])
        print("test! ", atomlist[0][0], atomlist[1][0])
        for residueID in np.unique(atomlist[0]):
            if residueID != resid:
                print(residueID, np.unique(mol.get("resname", sel="resid "+str(residueID))),)
        

            

[32 40 40 41 42] resid =  40
['OG' 'ND1' 'NE2' 'N' 'O']
test!  32 OG
32 ['SER']
41 ['PHE']
42 ['CYS']
[ 57  57  57 102 102 214] resid =  57
['N' 'ND1' 'NE2' 'OD1' 'OD2' 'O']
test!  57 N
102 ['ASP']
214 ['SER']
[91 91 91 92 93 94] resid =  91
['O' 'ND1' 'NE2' 'N' 'N' 'N']
test!  91 O
92 ['PRO']
93 ['SER']
94 ['TYR']


In [26]:
### This is for the NE2 atom
for resname, resid in zip(rn, rid):
    if resname == "HIS":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NE2 and resid "+str(resid))
        print(a, "resid", resid)
        for residueID in np.unique(a):
            print(residueID, np.unique(mol.get("resname", sel="resid "+str(residueID))), mol.get("name", sel="resid "+str(residueID)))

[ 40  40  42  42 193] resid 40
40 ['HIS'] ['N' 'CA' 'C' 'O' 'CB' 'CG' 'ND1' 'CD2' 'CE1' 'NE2']
42 ['CYS'] ['N' 'CA' 'C' 'O' 'CB' 'SG']
193 ['GLY'] ['N' 'CA' 'C' 'O']
[ 57  57 195 214] resid 57
57 ['HIS'] ['N' 'CA' 'C' 'O' 'CB' 'CG' 'ND1' 'CD2' 'CE1' 'NE2']
195 ['SER'] ['N' 'CA' 'C' 'O' 'CB' 'OG']
214 ['SER'] ['N' 'CA' 'C' 'O' 'CB' 'OG']
[91 91] resid 91
91 ['HIS'] ['N' 'CA' 'C' 'O' 'CB' 'CG' 'ND1' 'CD2' 'CE1' 'NE2']


In [23]:
mol.view()


Just realized proline can have an hidrogen in the nitrogen, and that Trp has a nitrogen that can be a donor. Article saying Cys sulfur is a moderately good hydrogen donor